In [6]:
!pip install minsearch


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [7]:
import minsearch
import json 

In [8]:
with open('documents.json', 'rt') as  f_in:
    docs_raw = json.load(f_in)

In [9]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index = minsearch.Index(
    text_fields=["questions", 'text', 'section'],
    keyword_fields=['course']
)

In [11]:
# SELECT * WHERE course = 'data-engineering-zoomcamp'

In [12]:
index.fit(documents)

In [13]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [14]:
import os
from openai import OpenAI
from dotenv import load_dotenv

base_url=os.environ.get("BASE_URL")
api_key=os.environ.get("API_KEY")

client = OpenAI(
    base_url=base_url,
    api_key=api_key,
)

In [20]:
print(base_url)

https://openrouter.ai/api/v1


In [15]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}


    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [16]:
def build_prompt(query, search_results):
    prompt_template = """
    Your are a course teaching assistant. 
    Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT  when answering the QUESTION. 
    If the CONTEXT doesn't containt the answer output None.

    QUESTION:
    {question}

    CONTEXT:
    {context}
    """.strip()
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion:{doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt 


In [17]:
def llm(prompt):
    responce = client.chat.completions.create(
        model="deepseek/deepseek-chat-v3-0324:free",
        messages=[{
            "role":"user",
            "content":prompt
        }]
    )
    return responce.choices[0].message.content

In [22]:
query = "how do I run kafka?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query=query, search_results=search_results)
    answer = llm(prompt=prompt)
    return answer

In [23]:
rag(query)

'To run Kafka, the context does not provide a direct step-by-step guide on how to start or set up Kafka. However, it does mention related troubleshooting steps and alternatives like Redpanda, which is compatible with Kafka APIs. \n\nIf you encounter specific errors while running Kafka (e.g., `NoBrokersAvailable`), the context suggests checking your Kafka broker Docker container with `docker ps` and running `docker compose up -d` in the folder containing your Docker Compose YAML file to start the instances. \n\nFor running Kafka-related Python scripts, the context provides installation instructions for dependencies like `confluent-kafka`, `fastavro`, or troubleshooting steps for errors (e.g., uninstalling/reinstalling `kafka-python==1.4.6`). \n\nIf you are looking for a general guide on setting up and running Kafka, the context does not contain that information. You may need to refer to external Kafka documentation. \n\nAnswer: None'

In [24]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

```bash
 docker run -it \
   --rm \
   --name elasticsearch \
   -m 4GB \
   -p 9200:9200 \
   -p 9300:9300 \
   -e "discovery.type=single-node" \
   -e "xpack.security.enabled=false" \
   elasticsearch:9.1.0

```

run elastic searrch with this docker code


In [25]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

In [26]:
es_client.info()

ObjectApiResponse({'name': 'a543528cee3c', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'QwZU7A_8QT6BZFaaCWwlkA', 'version': {'number': '9.1.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '00e7d33bf08f1476229d9d1642e2da46cfebdd53', 'build_date': '2025-07-23T22:09:53.891289976Z', 'build_snapshot': False, 'lucene_version': '10.2.2', 'minimum_wire_compatibility_version': '8.19.0', 'minimum_index_compatibility_version': '8.0.0'}, 'tagline': 'You Know, for Search'})

In [27]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [28]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [29]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:04<00:00, 206.35it/s]


In [30]:
query = 'I just disovered the course. Can I still join it?'


In [31]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [32]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [34]:
rag(query)


TypeError: 'NoneType' object is not subscriptable